# Processing of DOU masks

In [1]:
import nibabel as nib
import nilearn as nil
import numpy as np 
import sys
import pandas as pd
import re

import cmbnet.preprocessing.loading as loading

import cmbnet.preprocessing.process_masks as process_masks
from cmbnet.preprocessing.datasets.rodeja import load_RODEJA_data, perform_RODEJA_QC, load_RODEJA_raw
from cmbnet.preprocessing.loading import get_dataset_subjects
import cmbnet.visualization.utils_plotting as utils_plt
import os

import importlib
import cmbnet
importlib.reload(cmbnet)


<module 'cmbnet' from '/home/cerebriu/data/RESEARCH/MicrobleedNet/cmbnet/__init__.py'>

In [2]:
input_dir = "/home/cerebriu/data/datasets/raw/RODEJA"
dataset_name="rodeja"
subjects = loading.get_dataset_subjects(dataset_name, input_dir)

In [3]:
len(subjects)

103

In [4]:
df1 = pd.read_csv("/home/cerebriu/data/datasets/raw/RODEJA/cmb_annotations/notes_on_annotated_scans.txt", 
                    header=None, dtype=str,
                    sep = " ", 
                    names=['studyUID', 'n_CMB', "2remove"])
# df2 = pd.read_csv("/home/cerebriu/data/datasets/raw/RODEJA/cmb_annotations/notes_on_annotation.txt", 
#                   header=None, dtype=str,
#                   sep = " "
#                   )
df3 = pd.read_csv("/home/cerebriu/data/datasets/raw/RODEJA/cmb_annotations/notes_on_annotation_CMB.txt",
    header=None, dtype=str,
                    sep = " ",
                    names=['studyUID', 'n_CMB', "2remove"])

In [5]:
class Args:
    def __init__(self, input_dir = input_dir, dataset_name=dataset_name):
        self.input_dir = input_dir
        self.dataset_name = dataset_name
        self.plots_path = "/home/cerebriu/Downloads/"

In [6]:
args = Args()
results = {}
subjects_temp = subjects
subjects_temp = ["00006"]

for subject in subjects_temp:
    print("------------------------------------------------")
    print(subject)
    print("------------------------------------------------")

    # Load data

    # sequences_qc_f, labels_qc_f, metadata_out_F, seq_type, msg_f = load_RODEJA_data(args, subject, "")
    
    sequences_raw, labels_raw, sequence_type, cmb_info, msg = load_RODEJA_raw(args.input_dir, subject, "", "\t")
    print(f"CMBs: {cmb_info}")

    sequences_qc, labels_qc, labels_metadata, msg = perform_RODEJA_QC(sequences_raw, labels_raw, msg)

    mri_im_raw = sequences_raw[sequence_type]
    rawcmb_im_raw = labels_raw[sequence_type]
    grownmask_raw = labels_qc[sequence_type]

    results[subject] = {
        "raw": (sequences_raw, labels_raw, sequence_type, cmb_info),
        "qc": (sequences_qc, labels_qc, labels_metadata, msg)
    }

    print(f"shape: {mri_im_raw.shape}")

    com_list = [i['center_of_mass'] for i in cmb_info]

    # Plotting processed masks for each center of mass
    for i, CM in enumerate(com_list):
        print(CM)                                  
        metadata_str = f"""\
        sub: {subject}
        {sequence_type}, shape: {mri_im_raw.shape}
        CMBloc: {CM}   CMBsize: {labels_metadata["CMBs_old"][i]['size']}
        """


        filename_temp = os.path.join("/home/cerebriu/Downloads", f"RODEJA_{subject}-CMB_{i}.png")
        utils_plt.plot_processed_mask(mri_im_raw, rawcmb_im_raw, grownmask_raw, CM, 100, metadata_str=metadata_str, save_path=filename_temp)

------------------------------------------------
00006
------------------------------------------------
CMBs: [{'label': 1, 'size': 10, 'center_of_mass': (102, 130, 49)}, {'label': 2, 'size': 7, 'center_of_mass': (94, 118, 51)}, {'label': 3, 'size': 10, 'center_of_mass': (71, 107, 49)}, {'label': 4, 'size': 19, 'center_of_mass': (79, 151, 46)}, {'label': 5, 'size': 16, 'center_of_mass': (97, 174, 46)}, {'label': 6, 'size': 6, 'center_of_mass': (85, 158, 42)}, {'label': 7, 'size': 3, 'center_of_mass': (79, 152, 42)}, {'label': 8, 'size': 13, 'center_of_mass': (67, 155, 38)}, {'label': 9, 'size': 37, 'center_of_mass': (79, 117, 34)}, {'label': 10, 'size': 16, 'center_of_mass': (74, 55, 37)}, {'label': 11, 'size': 5, 'center_of_mass': (74, 60, 33)}, {'label': 12, 'size': 9, 'center_of_mass': (69, 58, 32)}, {'label': 13, 'size': 11, 'center_of_mass': (70, 70, 33)}, {'label': 14, 'size': 35, 'center_of_mass': (74, 125, 31)}, {'label': 15, 'size': 12, 'center_of_mass': (96, 123, 30)}, {'labe

KeyError: 54

In [ ]:
# for i, CM in enumerate(com_list):
#         print(CM)                                  
#         metadata_str = f"""\
#         sub: {subject}
#         {sequence_type}, shape: {mri_im_raw.shape}
#         CMBloc: {CM}   CMBsize: {labels_metadata[sequence_type]["CMBs_old"][i]['size']}
#         "connectivity": {labels_metadata[sequence_type]["CMBs_old"][i]['region_growing']['connectivity']}, "intensity_mode": {labels_metadata[sequence_type]["CMBs_old"][i]['region_growing']['intensity_mode']}, 
#         "diff_mode": {labels_metadata[sequence_type]["CMBs_old"][i]['region_growing']['diff_mode']}, 
#         "distance_th": {labels_metadata[sequence_type]["CMBs_old"][i]['region_growing']['distance_th']},"size_th": {labels_metadata[sequence_type]["CMBs_old"][i]['region_growing']['size_th']}"""


#         filename_temp = os.path.join("/home/cerebriu/Downloads", f"MOMENI_{subject}.png")
#         utils_plt.plot_processed_mask(mri_im_raw, rawcmb_im_raw, grownmask_raw, CM, 100, metadata_str=metadata_str, save_path=None)

NameError: name 'com_list' is not defined